In [7]:
'''monthly robberies data set'''
import pandas as pd
import math
# from sklearn.metrics import mean_squared_error
from math import sqrt
#load data set
data_series = pd.read_csv('./data/robberies.csv', header=0, index_col=0, parse_dates=True, squeeze=True)

#split data set into model development data set and cross validation set, use last year of data as cv
data_end = len(data_series) - 12
data_set, cv_set = data_series[0:data_end], data_series[data_end::]
print(f'Data set: {len(data_set)} months, Validation set: {len(cv_set)} months')

#save training and cv sets to csv format
data_set.to_csv('data_set.csv', header = False)
cv_set.to_csv('cv_set.csv', header = False)

Data set: 106 months, Validation set: 12 months


In [8]:
#create training set and test set
data = pd.read_csv('data_set.csv', header=None, index_col=0, parse_dates=True, squeeze=True)
X = data.values
X = X.astype('float32')

#start with train set of 50%
train_size = int(0.5*len(X))
train_set, test_set = X[0:train_size], X[train_size::]